# IMPORT

In [ ]:
import numpy as np 
import pandas as pd 
from pandas import Series

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
import arrow

import sklearn
from sklearn import metrics
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

from sklearn.model_selection import train_test_split, cross_val_score, KFold 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import auc, roc_auc_score, roc_curve

from catboost import CatBoostClassifier, Pool
from catboost.utils import get_roc_curve

from scipy.stats import boxcox
from scipy.stats import yeojohnson, uniform

# DATA

In [ ]:
df_train = pd.read_csv('../input/sf-dst-scoring/train.csv')
df_test = pd.read_csv('../input/sf-dst-scoring/test.csv')

df_train['sample'] = 1
df_test['sample'] = 0


# соединим тренировочный и тестовый датасеты
df = df_train.append(df_test)

In [ ]:
display(df_train.sample(3))
display(df_test.sample(3))

In [ ]:
print(df_train.shape)
print(df_test.shape)
print(df.shape)

In [ ]:
df.info()

In [ ]:
df.isna().sum()

Как видим, есть пропуски в столбце Образование

In [ ]:
df.columns

# EDA

In [ ]:
# Зададим списки числовых, категориальных и бинарных прихнаков
continous = ['age','score_bki', 'bki_request_cnt','decline_app_cnt', 'income']
categorical = ['home_address', 'work_address', 'sna', 'first_time', 'region_rating','education']
binary = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']

Колонку _App_date_ не включаем в списки, так как из нее будем отдельно делать признаки времени

## Continous

Посмотрим на распределение данных в числовых признаках

In [ ]:
for col in continous:
    plt.hist(df[col],bins=50,density=True, facecolor='g', alpha=0.75)
    plt.title(col)
    plt.grid(True)
    plt.show()

Как видим, только колонка 'score_bki' имеет нормальное распределение. К остальным попробуем применить трансформацию yeojohnson и логарифмирование.
Посмотрим, как это будет выглядеть

In [ ]:
shifted = ['income', 'age', 'bki_request_cnt', 'decline_app_cnt']

df_yj = pd.DataFrame()
df_lg = pd.DataFrame()

for col in shifted:
    df_yj[col] = yeojohnson(df[col])[0]
    df_lg[col] = np.log(df[col]+1)

In [ ]:
for col in shifted:
    
    fig, axes = plt.subplots(1, 2, figsize=(10, 5));
    sns.distplot(df_yj[col], kde = False, rug=False, ax = axes[0])
    sns.distplot(df_lg[col], kde = False, rug=False, ax = axes[1])
    plt.title(col)
    plt.show()

В целом преобразование с логарифмирование смотрится чуть получше.

In [ ]:
for col in shifted:
    df[col] = np.log(df[col]+1)

## Education

Посмотрим колонку eucation, где есть пропуски в данных

In [ ]:
df['education'].value_counts()

Наблюдается явный перевес SCH. Посему, пропуски в данных заполним именно SCH

In [ ]:
df['education'] = df['education'].fillna('SCH')

## Categorical

Заэнкодим категориальные переменные

In [ ]:
label_encoder = LabelEncoder()

for column in binary+categorical:
    df[column] = label_encoder.fit_transform(df[column])

In [ ]:
df.sample(3)

## Datetime

Спарсим дату и время

In [ ]:
df['app_date'] = df['app_date'].apply(lambda x: pd.to_datetime(x, format = '%d%b%Y'))
df['app_date_std'] = df['app_date'].apply(lambda x: arrow.get(x).timestamp)
df['app_date_std'] = df['app_date_std'].apply(lambda x: (x - df['app_date_std'].mean()) / df['app_date_std'].std())

In [ ]:
# сразу глянем, нет ли в наших признаках мультиколлинеарности

sns.heatmap(df[continous+['app_date_std']].corr().abs(), vmin=0, vmax=1)

In [ ]:
df.sample(3)

In [ ]:
df.columns

# CatBoost

In [ ]:
df_train_2 = df[df['sample']==1].drop(['client_id','sample'],axis=1)
df_test_2 = df[df['sample']==0].drop(['client_id','sample'],axis=1)

In [ ]:
df_train_2['default'].value_counts()

Наблюдаем дисбаланс целевой переменной

In [ ]:
# 
X = df_train_2.drop(['default'],axis=1)
y = df_train_2['default']

cat_features = binary + categorical
print('cat_features = ', cat_features)

train_pool = Pool(data=X, label=y, cat_features=cat_features)

In [ ]:
X_train, X_validation, y_train, y_validation = train_test_split(X, y, train_size=0.8, random_state=42)

Посмотрим, какое качество будет на дефолтных параметрах

In [ ]:

model = CatBoostClassifier(
    iterations=200,
    learning_rate=0.1,
    loss_function='Logloss', # используем logloss так как у нас задача классификации
    custom_loss=['AUC', 'F1']
)
model.fit(
    X_train, y_train,
    cat_features=cat_features,
    eval_set=(X_validation, y_validation),
    verbose=20,
    plot=True
)
print('Model is fitted: ' + str(model.is_fitted()))
print('Model params:')
print(model.get_params())

После 124 итерации уже пошло переобучение и catboost обрезал дерево

## Cross-val

In [ ]:
# теперь посмотрим, какую ошибку покажет нам кросс-валидация
from catboost import cv

params = {
    'loss_function': 'Logloss',
    'iterations': 100,
    'custom_loss': ['AUC', 'F1'],
    'random_seed': 42,
    'learning_rate': 0.15
}

cv_data = cv(
    params = params,
    pool = train_pool,
    fold_count=5,
    shuffle=True,
    partition_random_seed=0,
    plot=True,
    stratified=True,
    verbose=False
)

In [ ]:
# Посмотрим метрики на лучшей итерации

best_value = np.min(cv_data['test-Logloss-mean'])
best_iter = np.argmin(cv_data['test-Logloss-mean'])

print('Best validation Logloss score, not stratified: {:.4f}±{:.4f} on step {}'.format(
    best_value,
    cv_data['test-Logloss-std'][best_iter],
    best_iter)
)

In [ ]:
# Посмотрим на ROC кривую нашей модели


def print_roc(model):
    curve = get_roc_curve(model, train_pool)
    (fpr, tpr, thresholds) = curve
    roc_auc = sklearn.metrics.auc(fpr, tpr)


    plt.figure(figsize=(16, 8))
    lw = 2

    plt.plot(fpr, tpr, color='darkorange',
             lw=lw, label='ROC curve (area = %0.3f)' % roc_auc, alpha=0.5)

    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--', alpha=0.5)

    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xticks(fontsize=16)
    plt.yticks(fontsize=16)
    plt.grid(True)
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)
    plt.title('Receiver operating characteristic', fontsize=20)
    plt.legend(loc="lower right", fontsize=16)
    plt.show()

In [ ]:
print_roc(model)

In [ ]:
# теперь подберем адекватное для нашей задачи значение thresholds

from catboost.utils import get_fpr_curve
from catboost.utils import get_fnr_curve


curve = get_roc_curve(model, train_pool)
(thresholds, fpr) = get_fpr_curve(curve=curve) # значение ошибки 1 рода
(thresholds, fnr) = get_fnr_curve(curve=curve) # значение ошибки 2 рода

In [ ]:
plt.figure(figsize=(16, 8))
lw = 2

plt.plot(thresholds, fpr, color='blue', lw=lw, label='FPR', alpha=0.5)
plt.plot(thresholds, fnr, color='green', lw=lw, label='FNR', alpha=0.5)

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.grid(True)
plt.xlabel('Threshold', fontsize=16)
plt.ylabel('Error Rate', fontsize=16)
plt.title('FPR-FNR curves', fontsize=20)
plt.legend(loc="lower left", fontsize=16)
plt.show()

In [ ]:
'''
Так как для задачи банковского скоринга наиболее чувствительной является ошибка 2 рода 
(когда мы одобряем кредит клиенту, который в итоге обанкротится), то посмотрим, какой уровень
трешолд нам нужен, для снижения ошибки 2 рода
'''

from catboost.utils import select_threshold

print(select_threshold(model=model, data=train_pool, FNR=0.2))

### Влияние фичей

In [ ]:
feature_importance = model.get_feature_importance(prettified=True)
feature_importance

Очевидно, что самый важный параметр, это score_bki

# Submission

In [ ]:
predict_submission = model.predict_proba(df_test_2)
predict_submission

In [ ]:
for_ids = pd.read_csv('../input/sf-dst-scoring/test.csv')

In [ ]:
sample_submission = pd.DataFrame(columns = ['client_id','default'])
sample_submission['client_id'] = for_ids.client_id
sample_submission['default'] = 1- predict_submission
sample_submission

In [ ]:
sample_submission.to_csv('submission_3.csv', index=False)